# **Examples from https://doi.org/10.1016/j.aam.2020.102065** $\newcommand{\kdv}{\text{kdv}}$

In this notebook we will simulate the computations within the article https://doi.org/10.1016/j.aam.2020.102065 using the software `dalgebra`. This will be useful as a demonstration on how to use the software, a testing on the capabilities of the software and a chance to explore future features for the software.

For each example we will include the following:
* Small introduction to the problem.
* A cell wsetting up the framework for the example.
* A sequence of cells and texts developing the computations of the example.
* A last sentence with the conclusion or how to interpret the result.

In [1]:
import sys; sys.path.insert(0, "..") # dalgebra is here -- comment for installed version

%display latex
from dalgebra import *
from functools import lru_cache

## **1. Section 3**

### The sequence $\kdv_n$

In the article, the $\kdv_n$ sequence is defined recursively using two pseudo-differential operators:

$$\mathcal{R} = -\frac{1}{4}\partial^2 + u + \frac{1}{2} u' \partial^{-1},\qquad \mathcal{R}^* = -\frac{1}{4}\partial^2 + u - \frac{1}{2}\partial^{-1} u,$$

where $u$ is a differential indeterminate from a differential field $(C,\partial)$. The inverse operator $\partial^{-1}$ returns an antiderivative of the input to the operator. This can pose a serious problem because each time we apply it, we obtain a new different constant. Let see if we can express this directly on the system:

In [10]:
#B = DifferentialRing(InfinitePolynomialRing(QQ, "c"), lambda p : 0)

Unfortunately, the ring of infinitely many variables polynomial ring do not have a derivation module implemented. We can consider to implement one ourselves at some point, or simply try and avoid this by creating explicit variables for the integration constants when needed.

In [2]:
B = DifferentialRing(QQ, lambda p : 0) # change QQ to other polynomial ring with more constants if needed
R.<u> = DifferentialPolynomialRing(B)

The $\kdv_n$ operators are dfeined recursively:
$$\kdv_{n+1} = \mathcal{R}(\kdv_n),\qquad \kdv_0 = u'.$$
On the other hand, we can also define another recurrence using the adjoint operator $\mathcal{R}^*$:
$$v_{n+1} = \mathcal{R}^*(v_n),\qquad v_0 = 1.$$

It is shown in Equation (6) that these two sequences are related (due to the fact that $\mathcal{R}$ and $\mathcal{R}^*$ are adjoints):
$$2v_{n+1}' = \kdv_n.$$

In the following cell, we include two methods: the first tries to check whether a polynomial in $C\{u\}$ is a total derivative using a recursive elimination approach. The second method computes the application of $\mathcal{R}^*$ for any element $p(u) \in C\{u\}$ for which we know $u'p(u)$ is a total derivative (i.e., we know of some $a(u) \in C\{u\}$ such that $a'(u) = u'p(u)$.

In [3]:
def is_total_derivative(p, gen = None):
    # Getting variables from argument
    original = p
    R = p.parent(); B = R.base()
    if R.noperators() > 1: raise ValueError(f"Too many operators ({R.noperators()})")
    if not R.is_differential(): raise TypeError("Only differential fields are allowed")
    u = R.gen(gen if gen != None else 0)
    
    a = 0
    while(p != 0):
        #print("**********************************")
        #print(f"** Remaining polynomial: {p}\n-- Current antiderivative: {a}")
        order = p.order(u)
        #print(f"** Current order: {order}")
        if p.degree(u[order]) > 1: return False, f"The degree of the highest derivative is too big ({p.degree(u[p.order(u)])})"
    
        ## extracting parts
        highest_order = p.coefficient(u[order]); deg_order_1 = highest_order.degree(u[order-1])
        cs = [highest_order.coefficient(u[order-1]**i) for i in range(deg_order_1+1)]
        order_1_part = sum(cs[i]*u[order-1]**i for i in range(deg_order_1+1))
        q1 = highest_order - order_1_part # order q1 < d-1
        
        #print(f"    Order = d-1: {order_1_part}")
        #print(f"    Order < d-1: {q1}")
        new_a = sum(cs[i]*(1/(i+1)) * u[order-1]**(i+1) for i in range(deg_order_1+1)) + u[order-1]*q1
        p -= new_a.derivative()
        a += new_a
        
    assert a.derivative() == original
    return True, a

def apply_Rs(p, a, gen=0):
    r'''a' = u'*p'''
    R = p.parent(); u = R.gen(gen)
    new_p = -(1/4)*p.derivative(times=2) + u[0]*p - (1/2)*a
    is_total, new_a = is_total_derivative(u[1]*new_p, gen)
    if is_total:
        return new_p, new_a
    else:
        raise ValueError(f"u'P is not a total derivative: {new_a}")    
    return new_p, new_a

Hence, we can now compute the $\kdv_n$ sequence:

In [4]:
@lru_cache(maxsize=None)
def Rs(p,a,times=1):
    if times == 1:
        return apply_Rs(p,a)
    return Rs(*Rs(p,a,times=times-1))

@lru_cache(maxsize=None)
def v(n):
    return Rs(u._parent.one(), u[0], n)[0]

@lru_cache(maxsize=None)
def kdv(n):
    vn1 = v(n+1)
    return 2*vn1.derivative()

In [5]:
show(r"kdv(0) \longrightarrow " + latex(kdv(0)))
show(r"kdv(1) \longrightarrow " + latex(kdv(1)))
show(r"kdv(2) \longrightarrow " + latex(kdv(2)))
show(r"kdv(3) \longrightarrow " + latex(kdv(3)))

kdv(0) \longrightarrow 1 u_{1}

kdv(1) \longrightarrow \frac{3}{2} u_{1} u_{0} + \left(-\frac{1}{4}\right) u_{3}

kdv(2) \longrightarrow \frac{15}{8} u_{1} u_{0}^{2} + \left(-\frac{5}{8}\right) u_{3} u_{0} + \left(-\frac{5}{4}\right) u_{2} u_{1} + \frac{1}{16} u_{5}

kdv(3) \longrightarrow \frac{35}{16} u_{1} u_{0}^{3} + \left(-\frac{35}{32}\right) u_{3} u_{0}^{2} + \left(-\frac{35}{8}\right) u_{2} u_{1} u_{0} + \left(-\frac{35}{32}\right) u_{1}^{3} + \frac{7}{32} u_{5} u_{0} + \frac{21}{32} u_{4} u_{1} + \frac{35}{32} u_{3} u_{2} + \left(-\frac{1}{64}\right) u_{7}

### The Schrödinger operators

The $\kdv_n$ and $v_n$ sequences are useful to define some differential operators that will be related with the Schrödinger operator $L_u = L = -\partial^2 + u$. We see that to define this operator, we need two differential variables: $y$ will denote the solution to the operator and $u$ will be an arbitrary differential variable.

In [6]:
SchR.<u,y> = DifferentialPolynomialRing(QQ); show(SchR)
L = -y[2] + u[0]*y[0]; show(r"L \mapsto " + latex(L))

Ring of operator polynomials in (u, y) over Differential Ring [[Rational Field], (0,)]

L \mapsto 1 u_{0} y_{0} - y_{2}

For the operator $L_u$, we will define a sequence of odd order operators $P_{2n+1}$ that will commute with $L$. This sequence of operators are defined recursively using also the sequence $v_n$ and the operator $L$:

In [7]:
@lru_cache(maxsize=None)
def P_odd(n):
    r'''Computes P_{2n+1}'''
    if n == 0: # return $P_1$
        return y[1]
    else:
        vn = v(n)
        return vn*y[1] - (1/2)*vn.derivative()*y[0] + P_odd(n-1)(y=L)

In [8]:
show(r"P_1 \mapsto " + latex(P_odd(0)))
show(r"P_3 \mapsto " + latex(P_odd(1)))
show(r"P_5 \mapsto " + latex(P_odd(2)))
show(r"P_7 \mapsto " + latex(P_odd(3)))

P_1 \mapsto 1 y_{1}

P_3 \mapsto \frac{3}{4} u_{1} y_{0} + \frac{3}{2} u_{0} y_{1} - y_{3}

P_5 \mapsto \frac{15}{8} u_{1} u_{0} y_{0} + \frac{15}{8} u_{0}^{2} y_{1} + \left(-\frac{15}{16}\right) u_{3} y_{0} + \left(-\frac{25}{8}\right) u_{2} y_{1} + \left(-\frac{15}{4}\right) u_{1} y_{2} + \left(-\frac{5}{2}\right) u_{0} y_{3} + 1 y_{5}

P_7 \mapsto \frac{105}{32} u_{1} u_{0}^{2} y_{0} + \frac{35}{16} u_{0}^{3} y_{1} + \left(-\frac{105}{32}\right) u_{3} u_{0} y_{0} + \left(-\frac{105}{16}\right) u_{2} u_{1} y_{0} + \left(-\frac{175}{16}\right) u_{2} u_{0} y_{1} + \left(-\frac{245}{32}\right) u_{1}^{2} y_{1} + \left(-\frac{105}{8}\right) u_{1} u_{0} y_{2} + \left(-\frac{35}{8}\right) u_{0}^{2} y_{3} + \frac{63}{64} u_{5} y_{0} + \frac{161}{32} u_{4} y_{1} + \frac{175}{16} u_{3} y_{2} + \frac{105}{8} u_{2} y_{3} + \frac{35}{4} u_{1} y_{4} + \frac{7}{2} u_{0} y_{5} - y_{7}

It is here when the key property between the Schrödinger operator $L_u$ and the operators $P_{2n+1}$ comes into place (written in Lemma 3.2):
$$[P_{2n+1}, L] = \kdv_n$$

In [9]:
%%time
[
    (2*i+1,(P_odd(i)(y=L) - L(y=P_odd(i))) == kdv(i)*y[0]) # we divide by y_0 to see clearly the kdv formula from before.
    for i in range(10)
]

CPU times: user 1min 33s, sys: 184 ms, total: 1min 34s
Wall time: 1min 34s


[(1, True),
 (3, True),
 (5, True),
 (7, True),
 (9, True),
 (11, True),
 (13, True),
 (15, True),
 (17, True),
 (19, True)]

TODO: Continue after Lemma 3.2

### Example 3.3: Differential Sylvester resultant

In Example 3.3 of the article, a generic resultant is computed for two differential polynomials. We can reproduce that computation with this software:

In [24]:
T.<a0,a1,a2,b0,b1,b2,b3,d> = DifferentialPolynomialRing(QQ)
P1 = a2[0]*d[2] + a1[0]*d[1] + a0[0]*d[0]; P2 = b3[0]*d[3] + b2[0]*d[2] + b1[0]*d[1] + b0[0]*d[0]

In [26]:
P1_syl_P2 = P1.sylvester_resultant(P2, d)

In [31]:
for c,m in zip(P1_syl_P2.coefficients(), P1_syl_P2.monomials()):
    print(f"{m} --> {c}")

a0_2*a0_0*a2_0*b3_0^2 --> -1
a0_2*a1_1*a2_0*b3_0^2 --> -1
a0_2*a1_0^2*b3_0^2 --> 1
a0_2*a1_0*a2_1*b3_0^2 --> 1
a0_2*a1_0*a2_0*b2_0*b3_0 --> -1
a0_2*a2_0^2*b1_0*b3_0 --> 1
a0_1^2*a2_0*b3_0^2 --> 2
a0_1*a0_0*a1_0*b3_0^2 --> -3
a0_1*a0_0*a2_1*b3_0^2 --> -2
a0_1*a0_0*a2_0*b2_0*b3_0 --> 2
a0_1*a1_2*a2_0*b3_0^2 --> 1
a0_1*a1_1*a1_0*b3_0^2 --> -2
a0_1*a1_0^2*b2_0*b3_0 --> 1
a0_1*a1_0*a2_2*b3_0^2 --> -1
a0_1*a1_0*a2_1*b2_0*b3_0 --> 2
a0_1*a1_0*a2_0*b2_1*b3_0 --> 1
a0_1*a1_0*a2_0*b2_0^2 --> -1
a0_1*a1_0*a2_0*b2_0*b3_1 --> -1
a0_1*a2_1*a2_0*b1_0*b3_0 --> -2
a0_1*a2_0^2*b0_0*b3_0 --> -3
a0_1*a2_0^2*b1_1*b3_0 --> -1
a0_1*a2_0^2*b1_0*b2_0 --> 1
a0_1*a2_0^2*b1_0*b3_1 --> 1
a0_0^3*b3_0^2 --> 1
a0_0^2*a1_1*b3_0^2 --> 3
a0_0^2*a1_0*b2_0*b3_0 --> -1
a0_0^2*a2_2*b3_0^2 --> 1
a0_0^2*a2_1*b2_0*b3_0 --> -2
a0_0^2*a2_0*b1_0*b3_0 --> -2
a0_0^2*a2_0*b2_1*b3_0 --> -1
a0_0^2*a2_0*b2_0^2 --> 1
a0_0^2*a2_0*b2_0*b3_1 --> 1
a0_0*a1_2*a1_0*b3_0^2 --> -1
a0_0*a1_2*a2_1*b3_0^2 --> -1
a0_0*a1_2*a2_0*b2_0*b3_0 --> 1
a0_0